<a href="https://colab.research.google.com/github/2003Yash/huggingface-deployment-in-Sagemaker/blob/main/Huggingface_Sagemaker_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

create environment

In [ ]:
!pip install sagemaker -U # installs any updates happened to sagemaker version

create bucket

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()

sagemaker_session_bucket=None #  sagemaker session bucket -> used for uploading data, models and logs
                              # sagemaker will automatically create this bucket if it not exists


if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket() # set to default bucket if a bucket name is not given

Role Mangement

In [ ]:
try:
    role = sagemaker.get_execution_role() # try calling role directly

except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn'] # it direct call fails use this code

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}") # we use roles to call models
print(f"sagemaker session region: {sess.boto_region_name}")

Import Model from HF-Hub

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

# Hub model configuration <https://huggingface.co/models>
hub = {
  'HF_MODEL_ID':'distilbert-base-uncased-distilled-squad', # model_id from hf.co/models
  'HF_TASK':'question-answering'                           # NLP task you want to use for predictions
}

Create model from imported Model along with environment config

In [ ]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,                                                # configuration for loading model from Hub
   role=role,                                              # IAM role with permissions to create an endpoint
   transformers_version="4.26",                             # Transformers version used
   pytorch_version="1.13",                                  # PyTorch version used
   py_version='py39',                                      # Python version used
)

Deploy Model

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

# O/P = -----!
# each - in output defines avg minute taken to deploy it and it usually takes 6-7 minutes

In [ ]:
# example request: you always need to define "inputs"
data = {
"inputs": {
	"question": "What is used for inference?",
	"context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
	}
}

In [ ]:
predictor.predict(data)

In [ ]:
# O/P = {'score': 0.9987204670906067, 'start': 68, 'end': 77, 'answer': 'sagemaker'}

new question

In [ ]:
data = {
"inputs": {
	"question": "What does Krish teach?",
	"context": "My Name is Krish and  I teach data science."
	}
}

In [ ]:
predictor.predict(data)

In [ ]:
# O/P = {'score': 0.9854100942611694, 'start': 30, 'end': 42, 'answer': 'data science'}

**Endpoints are visible in Sagemaker -> deployments -> End points**

Clean the endponits and buckets

In [ ]:
MANUNALLY DELETE THE BUCKET INSIDE S3
SELECT NOTEBOOK INSTANCE -> STOP
-> (AFTER IT STOPS) DELET